In [1]:
__author__ = "Christopher Potts"
__version__ = "CS224u, Stanford, Spring 2020"

## Contents

1. [Overview](#Overview)
1. [Set-up](#Set-up)
1. [All two-word examples as a dev corpus](#All-two-word-examples-as-a-dev-corpus)
1. [Dev dataset](#Dev-dataset)
1. [Random train–test split for development](#Random-train–test-split-for-development)
1. [Question 1: Improve the tokenizer [1 point]](#Question-1:-Improve-the-tokenizer-[1-point])
1. [Use the tokenizer](#Use-the-tokenizer)
1. [Question 2: Improve the color representations [1 point]](#Question-2:-Improve-the-color-representations-[1-point])
1. [Use the color representer](#Use-the-color-representer)
1. [Initial model](#Initial-model)
1. [Question 3: GloVe embeddings [1 points]](#Question-3:-GloVe-embeddings-[1-points])
1. [Try the GloVe representations](#Try-the-GloVe-representations)
1. [Question 4: Color context [3 points]](#Question-4:-Color-context-[3-points])
1. [Your original system [3 points]](#Your-original-system-[3-points])
1. [Bakeoff [1 point]](#Bakeoff-[1-point])

## Overview

This homework and associated bake-off are oriented toward building an effective system for generating color descriptions that are pragmatic in the sense that they would help a reader/listener figure out which color was being referred to in a shared context consisting of a target color (whose identity is known only to the describer/speaker) and a set of distractors.

The notebook [colors_overview.ipynb](colors_overview.ipynb) should be studied before work on this homework begins. That notebook provides backgroud on the task, the dataset, and the modeling code that you will be using and adapting.

The homework questions are more open-ended than previous ones have been. Rather than asking you to implement pre-defined functionality, they ask you to try to improve baseline components of the full system in ways that you find to be effective. As usual, this culiminates in a prompt asking you to develop a novel system for entry into the bake-off. In this case, though, the work you do for the homework will likely be directly incorporated into that system.

## Set-up

See [colors_overview.ipynb](colors_overview.ipynb) for set-up in instructions and other background details.

In [1]:
from colors import ColorsCorpusReader
import os
from sklearn.model_selection import train_test_split
from torch_color_describer import (
    ContextualColorDescriber, create_example_dataset)
import utils
from utils import START_SYMBOL, END_SYMBOL, UNK_SYMBOL

In [2]:
utils.fix_random_seeds()

In [3]:
COLORS_SRC_FILENAME = os.path.join(
    "data", "colors", "filteredCorpus.csv")

## All two-word examples as a dev corpus

So that you don't have to sit through excessively long training runs during development, I suggest working with the two-word-only subset of the corpus until you enter into the late stages of system testing.

In [4]:
dev_corpus = ColorsCorpusReader(
    COLORS_SRC_FILENAME, 
    word_count=2, 
    normalize_colors=True)

In [5]:
dev_examples = list(dev_corpus.read())

This subset has about one-third the examples of the full corpus:

In [6]:
len(dev_examples)

13890

We __should__ worry that it's not a fully representative sample. Most of the descriptions in the full corpus are shorter, and a large proportion are longer. So this dataset is mainly for debugging, development, and general hill-climbing. All findings should be validated on the full dataset at some point.

## Dev dataset

The first step is to extract the raw color and raw texts from the corpus:

In [7]:
dev_rawcols, dev_texts = zip(*[[ex.colors, ex.contents] for ex in dev_examples])

The raw color representations are suitable inputs to a model, but the texts are just strings, so they can't really be processed as-is. Question 1 asks you to do some tokenizing!

## Random train–test split for development

For the sake of development runs, we create a random train–test split:

In [8]:
dev_rawcols_train, dev_rawcols_test, dev_texts_train, dev_texts_test = \
    train_test_split(dev_rawcols, dev_texts)

## Question 1: Improve the tokenizer [1 point]

This is the first required question – the first required modification to the default pipeline.

The function `tokenize_example` simply splits its string on whitespace and adds the required start and end symbols:

In [17]:
def tokenize_example(s):
    
    # Improve me!
    from nltk.tokenize.treebank import TreebankWordTokenizer, TreebankWordDetokenizer
    t = TreebankWordTokenizer()
    toks = [START_SYMBOL] + t.tokenize(s)+ [END_SYMBOL]
    return toks
    #return [START_SYMBOL] + s.split() + [END_SYMBOL]
    

In [18]:
tokenize_example(dev_texts_train[376])

['<s>', 'aqua', ',', 'teal', '</s>']

__Your task__: Modify `tokenize_example` so that it does something more sophisticated with the input text. 

__Notes__:

* There are useful ideas for this in [Monroe et al. 2017](https://transacl.org/ojs/index.php/tacl/article/view/1142)
* There is no requirement that you do word-level tokenization. Sub-word and multi-word are options.
* This question can interact with the size of your vocabulary (see just below), and in turn with decisions about how to use `UNK_SYMBOL`.

__Important__: don't forget to add the start and end symbols, else the resulting models will definitely be terrible!

## Use the tokenizer

Once the tokenizer is working, run the following cell to tokenize your inputs:

In [19]:
dev_seqs_train = [tokenize_example(s) for s in dev_texts_train]

dev_seqs_test = [tokenize_example(s) for s in dev_texts_test]

We use only the train set to derive a vocabulary for the model:

In [20]:
dev_vocab = sorted({w for toks in dev_seqs_train for w in toks}) + [UNK_SYMBOL]

It's important that the `UNK_SYMBOL` is included somewhere in this list. Test examples with word not seen in training will be mapped to `UNK_SYMBOL`. If you model's vocab is the same as your train vocab, then `UNK_SYMBOL` will never be encountered during training, so it will be a random vector at test time.

In [21]:
len(dev_vocab)

1302

## Question 2: Improve the color representations [1 point]

This is the second required pipeline improvement for the assignment. 

The following functions do nothing at all to the raw input colors we get from the corpus. 

In [51]:
def represent_color_context(colors):
    import numpy as np
    # Improve me!
    # https://arxiv.org/pdf/1606.03821.pdf
    color_fft = np.fft.fftn(colors, (2,27))
    #return [represent_color(color) for color in colors]
    return color_fft

def represent_color(color):
    import numpy as np
    # Improve me!
    color_fft = np.fft.fftn(color, axes=(1))
    #return color
    return color_fft

In [52]:
represent_color_context(dev_rawcols_train[0])

array([[ 2.32166667+0.j        ,  2.25429194-0.40115956j,
         2.06221957-0.75360599j,  1.77369742-1.01701456j,
         1.42984003-1.16615921j,  1.07697461-1.19461809j,
         0.75823165-1.11477541j,  0.50609126-0.95419285j,
         0.33737807-0.7491761j ,  0.25166667-0.53693575j,
         0.23332999-0.34801654j,  0.25668917-0.20058686j,
         0.29307093-0.09776085j,  0.31818536-0.02845888j,
         0.31818536+0.02845888j,  0.29307093+0.09776085j,
         0.25668917+0.20058686j,  0.23332999+0.34801654j,
         0.25166667+0.53693575j,  0.33737807+0.7491761j ,
         0.50609126+0.95419285j,  0.75823165+1.11477541j,
         1.07697461+1.19461809j,  1.42984003+1.16615921j,
         1.77369742+1.01701456j,  2.06221957+0.75360599j,
         2.25429194+0.40115956j],
       [-0.71277778+0.j        , -0.695759  +0.07180787j,
        -0.64832315+0.12833971j, -0.58056149+0.15756924j,
        -0.50688926+0.15327832j, -0.44297912+0.11637978j,
        -0.40242696+0.05472322j, -0.39

In [54]:
represent_color_context([[0.625, 0.5, 0.8]])

array([[ 1.925     +0.j        ,  1.82642855-0.47434728j,
         1.54954319-0.86609814j,  1.14694076-1.10924001j,
         0.69413671-1.16745321j,  0.27404657-1.04100697j,
        -0.03993001-0.76601999j, -0.1986631 -0.40627975j,
        -0.18679187-0.03938757j, -0.025     +0.25980762j,
         0.23536332+0.43495971j,  0.52411991+0.4598184j ,
         0.76798924+0.34322002j,  0.90681672+0.12644624j,
         0.90681672-0.12644624j,  0.76798924-0.34322002j,
         0.52411991-0.4598184j ,  0.23536332-0.43495971j,
        -0.025     -0.25980762j, -0.18679187+0.03938757j,
        -0.1986631 +0.40627975j, -0.03993001+0.76601999j,
         0.27404657+1.04100697j,  0.69413671+1.16745321j,
         1.14694076+1.10924001j,  1.54954319+0.86609814j,
         1.82642855+0.47434728j],
       [ 1.925     +0.j        ,  1.82642855-0.47434728j,
         1.54954319-0.86609814j,  1.14694076-1.10924001j,
         0.69413671-1.16745321j,  0.27404657-1.04100697j,
        -0.03993001-0.76601999j, -0.19

__Your task__: Modify `represent_color_context` and/or `represent_color` to represent colors in a new way.
    
__Notes__:

* The Fourier-transform method of [Monroe et al. 2017](https://transacl.org/ojs/index.php/tacl/article/view/1142) is a proven choice.
* You are not required to keep `represent_color`. This might be unnatural if you want to perform an operation on each color trio all at once.
* For that matter, if you want to process all of the color contexts in the entire data set all at once, that is fine too, as long as you can also perform the operation at test time with an unknown number of examples being tested.

## Use the color representer

The following cell just runs your `represent_color_context` on the train and test sets:

In [55]:
dev_cols_train = [represent_color_context(colors) for colors in dev_rawcols_train]

dev_cols_test = [represent_color_context(colors) for colors in dev_rawcols_test]

At this point, our preprocessing steps are complete, and we can fit a first model.

## Initial model

The first model is configured right now to be a small model run for just a few iterations. It should be enough to get traction, but it's unlikely to be a great model. You are free to modify this configuration if you wish; it is here just for demonstration and testing:

In [56]:
dev_mod = ContextualColorDescriber(
    dev_vocab, 
    embed_dim=10, 
    hidden_dim=10, 
    max_iter=5, 
    batch_size=128)

In [57]:
_ = dev_mod.fit(dev_cols_train, dev_seqs_train)

/Users/simranjit/Neuron/repos/cs22u_fork/torch_color_describer.py:75: ComplexWarning: Casting complex values to real discards the imaginary part
  color_seqs = torch.FloatTensor(color_seqs)
Epoch 5; err = 126.50223267078495

As discussed in [colors_overview.ipynb](colors_overview.ipynb), our primary metric is `listener_accuracy`:

In [58]:
dev_mod.listener_accuracy(dev_cols_test, dev_seqs_test)

0.5643535847970055

We can also see the model's predicted sequences given color context inputs:

In [59]:
dev_mod.predict(dev_cols_test[:1])

/Users/simranjit/Neuron/repos/cs22u_fork/torch_color_describer.py:439: ComplexWarning: Casting complex values to real discards the imaginary part
  color_seqs = torch.FloatTensor(color_seqs)


[['<s>', 'bright', 'green', '</s>']]

In [60]:
dev_seqs_test[:1]

[['<s>', 'right', 'side', '#', '#', '#', 'purple', 'pinkish', '</s>']]

## Question 3: GloVe embeddings [1 points]

The above model uses a random initial embedding, as configured by the decoder used by `ContextualColorDescriber`. This homework question asks you to consider using GloVe inputs. 

__Your task__: Complete `create_glove_embedding` so that it creates a GloVe embedding based on your model vocabulary. This isn't mean to be analytically challenging, but rather just to create a basis for you to try out other kinds of rich initialization.

In [61]:
GLOVE_HOME = os.path.join('data', 'glove.6B')

In [62]:
def create_glove_embedding(vocab, glove_base_filename='glove.6B.50d.txt'):
    
    # Use `utils.glove2dict` to read in the GloVe file:    
    ##### YOUR CODE HERE

    glove_src = os.path.join(GLOVE_HOME, glove_base_filename)
    GLOVE = utils.glove2dict(glove_src)
    
    # Use `utils.create_pretrained_embedding` to create the embedding.
    # This function will, by default, ensure that START_TOKEN, 
    # END_TOKEN, and UNK_TOKEN are included in the embedding.
    ##### YOUR CODE HERE
    """
    Parameters
    ----------
    lookup : dict
        Must map words to their vector representations.
    vocab : list of str
        Words to create embeddings for.
    required_tokens : tuple of str
        Tokens that must have embeddings. If they are not available
        in the look-up, they will be given random representations.
        
     Returns
    -------
    np.array, list
        The np.array is an embedding for `vocab` and the `list` is
        the potentially expanded version of `vocab` that came in.

    """
    embedding, expanded_vocab = utils.create_pretrained_embedding(GLOVE, vocab)

    
    # Be sure to return the embedding you create as well as the
    # vocabulary returned by `utils.create_pretrained_embedding`,
    # which is likely to have been modified from the input `vocab`.
    
    ##### YOUR CODE HERE
    return embedding, expanded_vocab



## Try the GloVe representations

Let's see if GloVe helped for our development data:

In [63]:
dev_glove_embedding, dev_glove_vocab = create_glove_embedding(dev_vocab)

The above might dramatically change your vocabulary, depending on how many items from your vocab are in the Glove space:

In [64]:
len(dev_vocab)

1302

In [65]:
len(dev_glove_vocab)

1302

In [66]:
dev_mod_glove = ContextualColorDescriber(
    dev_glove_vocab, 
    embedding=dev_glove_embedding,
    hidden_dim=10, 
    max_iter=5, 
    batch_size=128)

In [67]:
_ = dev_mod_glove.fit(dev_cols_train, dev_seqs_train)

Epoch 5; err = 143.95896708965302

In [68]:
dev_mod_glove.listener_accuracy(dev_cols_test, dev_seqs_test)

0.4938093866973798

In [69]:
print(dev_glove_embedding.shape)

(1302, 50)


You probably saw a small boost, assuming your tokeization scheme leads to good overlap with the GloVe vocabulary. The input representations are larger than in our previous model (at least as I configured things), so we would need to do more runs with higher `max_iter` values to see whether this is worthwhile overall.

## Question 4: Color context [3 points]

The final required homework question is the most challenging, but it should set you up to think in much more flexible ways about the underlying model we're using.

The question asks you to modify various model components in `torch_color_describer.py`. The section called [Modifying the core model](colors_overview.ipynb#Modifying-the-core-model) from the core unit notebook provides a number of examples illustrating the basic techniques, so you might review that material if you get stuck here.

__Your task__: [Monroe et al. 2017](https://transacl.org/ojs/index.php/tacl/article/view/1142) append the target color (the final one in the context) to each input token that gets processed by the decoder. The question asks you to subclass the `Decoder` and `EncoderDecoder` from `torch_color_describer.py` so that you can build models that do this.

__Step 1__: Modify the `Decoder` so that the input vector to the model at each timestep is not just a token representaton `x` but the concatenation of `x` with the representation of the target color.

__Notes__:

* You might notice at this point that the original `Decoder.forward` method has an optional keyword argument `target_colors` that is passed to `Decoder.get_embeddings`. Because this is already in place, all you have to do is modify the `get_embeddings` method to use this argument.

* The change affects the configuration of `self.rnn`, so you need to subclass the `__init__` method as well, so that its `input_size` argument accomodates the embedding as well as the color representations.

* You can do the relevant operations efficiently in pure PyTorch using `repeat_interleave` and `cat`, but the important thing is to get a working implementation – you can always optimize the code later if the ideas prove useful to you. 

Here's skeleton code for you to flesh out:

In [105]:
import torch
import torch.nn as nn
from torch_color_describer import Decoder

class ColorContextDecoder(Decoder):    
    def __init__(self, color_dim, *args, **kwargs):
        self.color_dim = color_dim
        super().__init__(*args, **kwargs)
        # print("ColorContextDecoder color_dim", color_dim )
        
        # Fix the `self.rnn` attribute:
        ##### YOUR CODE HERE
        
        # decoder params  vocab_size, embed_dim, hidden_dim, embedding=None
        
        """
        You might notice at this point that the original Decoder.forward method has an optional
        keyword argument target_colors that is passed to Decoder.get_embeddings. 
        Because this is already in place, all you have to do is modify the get_embeddings method to use this argument.
        The change affects the configuration of self.rnn, so you need to subclass the __init__ method as well,
        so that its input_size argument accomodates the embedding as well as the color representations.
        """
        
        
        self.legacy = False
        delta = self.color_dim
        #if self.legacy:
            #delta = 0
        #print("self.embed_dim+delta", (self.embed_dim+delta))
        self.rnn = nn.GRU(
            input_size=self.embed_dim+delta,
            hidden_size=self.hidden_dim,
            batch_first=True)

    def get_embeddings(self, word_seqs, target_colors=None):  
        """You can assume that `target_colors` is a tensor of shape 
        (m, n), where m is the length of the batch (same as 
        `word_seqs.shape[0]`) and n is the dimensionality of the 
        color representations the model is using. The goal is
        to attached each color vector i to each of the tokens in
        the ith sequence of (the embedded version of) `word_seqs`.
        
        """        
        ##### YOUR CODE HERE
        """ 
        Expected embeddings: You should get the word embeddings for each word and
        concatenate each of these with your color embedding. 
        Definitely don't do this with the incoming word indices, and make sure that the concatenation 
        is true concatenation -- it shouldn't change the order (in the sense of 1d, 2d, etc.) of the tensors.
        """ 
        """
        m = word_seqs.shape[0] = batch_size
        n = color emebedding
        """
        # Your task: Monroe et al. 2017 append the target color
        # (the final one in the context) to each input token that gets processed by the decoder. 
        embeddings = self.embedding(word_seqs)
        #dev_glove_embedding
        """print("word_seqs.shape", word_seqs.shape)
        print("embeddings.shape", embeddings.shape)
        print("target_colors.shape", target_colors.shape)
        word_seqs.shape torch.Size([112, 6])
        embeddings.shape torch.Size([112, 6, 10])
        target_colors.shape torch.Size([112, 5])
        """
        embeddings_shape = embeddings.shape
        #print("target_colors.shape", target_colors.type())
        targs = torch.repeat_interleave(target_colors.float(), embeddings_shape[1], dim=1)
        targs = targs.reshape(embeddings_shape[0], embeddings_shape[1], target_colors.shape[-1])
        #print("embeddings.shape", embeddings.type())
        #print("targs.shape", targs.type())
        
        concat_op = torch.cat((embeddings,targs ), -1)
        #if self.legacy:
        #    concat_op = embeddings
        #print("concat_op.shape", concat_op.shape)
        return concat_op

__Step 2__: Modify the `EncoderDecoder`. For this, you just need to make a small change to the `forward` method: extract the target colors from `color_seqs` and feed them to the decoder.

In [106]:
from torch_color_describer import EncoderDecoder

class ColorizedEncoderDecoder(EncoderDecoder):
    
    def forward(self, 
            color_seqs, 
            word_seqs, 
            seq_lengths=None, 
            hidden=None, 
            targets=None):
        if hidden is None:
            hidden = self.encoder(color_seqs)
            
        # Extract the target colors from `color_seqs` and 
        # feed them to the decoder, which already has a
        # `target_colors` keyword.        
        ##### YOUR CODE HERE
        # forward(self, word_seqs, seq_lengths=None, hidden=None, target_colors=None):
        color_seqs_shape = color_seqs.shape
        #print("color_seqs.shape", color_seqs_shape)
        # color_seqs.shape torch.Size([112, 3, 2])
        # https://us.edstem.org/courses/326/discussion/59721
        
        #print("color_seqs_shape.shape", color_seqs_shape)
        mid_index = color_seqs_shape[1]-1
        extracted_targets = color_seqs[:,2]
        #print("extracted_targets.shape", extracted_targets.shape)
        #print("hidden_ColorizedEncoderDecoder ", hidden)
        #print(type(word_seqs), type(hidden), type(extracted_targets))
        output, hidden = self.decoder(word_seqs, hidden=hidden,seq_lengths=seq_lengths, target_colors=extracted_targets)
        
        return output, hidden, targets

__Step 3__: Finally, as in the examples in [Modifying the core model](colors_overview.ipynb#Modifying-the-core-model), you need to modify the `build_graph` method of `ContextualColorDescriber` so that it uses your new `ColorContextDecoder` and `ColorizedEncoderDecoder`. Here's starter code:

In [107]:
from torch_color_describer import Encoder

class ColorizedInputDescriber(ContextualColorDescriber):
        
    def build_graph(self):
        
        # We didn't modify the encoder, so this is
        # just copied over from the original:
        encoder = Encoder(
            color_dim=self.color_dim,
            hidden_dim=self.hidden_dim)

        # Use your `ColorContextDecoder`, making sure
        # to pass in all the keyword arguments coming
        # from `ColorizedInputDescriber`:
        
        ##### YOUR CODE HERE
        
        """ hidden dims"""
        # color_dim,  vocab_size, embed_dim, hidden_dim, embedding=None
         # ColorContextDecoder-> Decoder->
        """
        vocab_size : int
        embed_dim : int
        hidden_dim : int
        embedding : np.array or None
        If `None`, a random embedding is created. If `np.array`, this
        value becomes the embedding.
        
        """
        decoder = ColorContextDecoder(self.color_dim, self.vocab_size, self.embed_dim, self.hidden_dim,self.embedding)
               
        # Return a `ColorizedEncoderDecoder` that uses
        # your encoder and decoder:
        
        ##### YOUR CODE HERE
        colorized_encoder_decoder = ColorizedEncoderDecoder(encoder, decoder)
        return colorized_encoder_decoder
    


That's it! Since these modifications are pretty intricate, you might want to use [a toy dataset](colors_overview.ipynb#Toy-problems-for-development-work) to debug it:

In [108]:
toy_color_seqs, toy_word_seqs, toy_vocab = create_example_dataset(
    group_size=50, vec_dim=2)

In [109]:
toy_color_seqs_train, toy_color_seqs_test, toy_word_seqs_train, toy_word_seqs_test = \
    train_test_split(toy_color_seqs, toy_word_seqs)

In [110]:
toy_mod = ColorizedInputDescriber(
    toy_vocab, 
    embed_dim=10, 
    hidden_dim=10, 
    max_iter=100, 
    batch_size=128)

In [111]:
_ = toy_mod.fit(toy_color_seqs_train, toy_word_seqs_train)

Epoch 100; err = 0.11660385131835938

In [112]:
toy_mod.listener_accuracy(toy_color_seqs_test, toy_word_seqs_test)

1.0

If that worked, then you can now try this model on SCC problems!

## Your original system [3 points]

There are many options for your original system, which consists of the full pipeline – all preprocessing and modeling steps. You are free to use any model you like, as long as you subclass `ContextualColorDescriber` in a way that allows its `listener_accuracy` method to behave in the expected way.

So that we can evaluate models in a uniform way for the bake-off, we ask that you modify the function `my_original_system` below so that it accepts a trained instance of your model and does any preprocessing steps required by your model.

If we seek to reproduce your results, we will rerun this entire notebook. Thus, it is fine if your `my_original_system` makes use of functions you wrote or modified above this cell.

In [117]:
def my_original_system(trained_model, color_seqs_test, texts_test): 
    """Feel free to modify this code to accommodate the needs of
    your system. Just keep in mind that it will get raw corpus
    examples as inputs for the bake-off.
    
    """    
    # `word_seqs_test` is a list of strings, so tokenize each of
    # its elements:    
    tok_seqs = [tokenize_example(s) for s in texts_test]
    
    col_seqs = [represent_color_context(colors) 
                for colors in color_seqs_test]

    # Return the `listener_accuracy` for your model:
    return trained_model.listener_accuracy(col_seqs, tok_seqs)

In [124]:
class MYColorizedInputDescriber(ContextualColorDescriber):
        
    def build_graph(self):
        
        # We didn't modify the encoder, so this is
        # just copied over from the original:
        encoder = Encoder(
            color_dim=self.color_dim,
            hidden_dim=self.hidden_dim)

        # Use your `ColorContextDecoder`, making sure
        # to pass in all the keyword arguments coming
        # from `ColorizedInputDescriber`:
        
        ##### YOUR CODE HERE
        
        """ hidden dims"""
        # color_dim,  vocab_size, embed_dim, hidden_dim, embedding=None
         # ColorContextDecoder-> Decoder->
        """
        vocab_size : int
        embed_dim : int
        hidden_dim : int
        embedding : np.array or None
        If `None`, a random embedding is created. If `np.array`, this
        value becomes the embedding.
        
        """
        decoder = ColorContextDecoder(self.color_dim, self.vocab_size, self.embed_dim, self.hidden_dim,self.embedding)
               
        # Return a `ColorizedEncoderDecoder` that uses
        # your encoder and decoder:
        
        ##### YOUR CODE HERE
        colorized_encoder_decoder = ColorizedEncoderDecoder(encoder, decoder)
        return colorized_encoder_decoder
    
#bake_off_model =     
"""bake_off_model = MYColorizedInputDescriber(
    dev_vocab, 
    embed_dim=10, 
    hidden_dim=10, 
    max_iter=100, 
    batch_size=128)
 """
bake_off_model = MYColorizedInputDescriber(
                    dev_vocab, 
                    embedding=None,  # Option to supply a pretrained matrix as an `np.array`.
                    embed_dim=10, 
                    hidden_dim=10, 
                    max_iter=3, 
                    eta=0.01,
                    optimizer=torch.optim.Adam,
                    batch_size=128,
                    l2_strength=0.0,
                    warm_start=False,
                    device=None)
#toy_mod.fit(toy_color_seqs_train, toy_word_seqs_train)
bake_off_model.fit(dev_rawcols_train, dev_texts_train)

Epoch 3; err = 230.64733958244324

MYColorizedInputDescriber(
	hidden_dim=10,
	batch_size=128,
	max_iter=3,
	eta=0.01,
	optimizer=<class 'torch.optim.adam.Adam'>,
	l2_strength=0.0,
	embed_dim=10,
	embedding=None)

If `my_original_system` works on test sets you create from the corpus distribution, then it will works for the bake-off, so consider checking that. For example, this would check that `dev_mod` above passes muster:

In [125]:
my_original_system(dev_mod, dev_rawcols_test, dev_texts_test)

0.5643535847970055

In the cell below, please provide a brief technical description of your original system, so that the teaching team can gain an understanding of what it does. This will help us to understand your code and analyze all the submissions to identify patterns and strategies.

In [ ]:
# Enter your system description in this cell.
# Please do not remove this comment.



## Bakeoff [1 point]

For the bake-off, we will release a test set. The announcement will go out on the discussion forum. You will evaluate your custom model from the previous question on these new datasets using your `my_original_system` function. Rules:

1. Only one evaluation is permitted.
1. No additional system tuning is permitted once the bake-off has started.

The cells below this one constitute your bake-off entry.

People who enter will receive the additional homework point, and people whose systems achieve the top score will receive an additional 0.5 points. We will test the top-performing systems ourselves, and only systems for which we can reproduce the reported results will win the extra 0.5 points.

Late entries will be accepted, but they cannot earn the extra 0.5 points. Similarly, you cannot win the bake-off unless your homework is submitted on time.

The announcement will include the details on where to submit your entry.

In [ ]:
# Enter your bake-off assessment code in this cell. 
# Please do not remove this comment.


In [ ]:
# On an otherwise blank line in this cell, please enter
# your listener_accuracy score as reported by the code
# above. Please enter only a number between 0 and 1 inclusive. 
# Please do not remove this comment.
